In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
ASSIGN_TREAT = 0.8
df = pd.read_csv("MINIMAL_FOLLOWERS_02.19.2024__17.01.30__START0_END-1_uofmisinfowatch_acresearcher.csv", dtype={'followers_id':'object'})

In [3]:
def make_group_files(tdf):
    spreaders = list(tdf['main'].unique())
    for spreader in spreaders:
        spreader_df = tdf[tdf['main'] == spreader]
        spreader_treat = spreader_df.query("treated==1")
        spreader_ctrl = spreader_df.query("treated==0")
        spreader_treat[['followers_id']].to_csv(f"treat_{spreader.lower()}.txt", index=False, header=None)
        spreader_ctrl[['followers_id']].to_csv(f"ctrl_{spreader.lower()}.txt", index=False, header=None)

def assign_group(sub_df, treat_prob=ASSIGN_TREAT):
    np.random.seed(42)  
    random.seed(42)
    sub_df['group'] = np.random.choice(['treatment', 'control'], size=len(sub_df), p=[treat_prob, 1-treat_prob])
    return sub_df
    
def print_stats(df):
    print("PRINTING STATS")
    print("Total N edges:", len(df))
    print("Followers by spreader:")
    print(df.groupby(by=['main']).count().reset_index().sort_values(by=['followers_id']))
    
    
print_stats(df)
df = df.groupby('main').apply(assign_group).reset_index(drop=True)
df['treated'] = df['group'].apply(lambda x: 1 if x=='treatment' else 0)

print("\nSplitting into treatment and control with treat prob = {}".format(ASSIGN_TREAT))
print("Verify logic works:")
print(df.groupby(by=['main'])['group'].value_counts(normalize=True))
    
print("\nRaw counts")
print(df.groupby(by=['main'])['group'].value_counts(normalize=False))

make_group_files(df)

df.to_csv("treat_status_MINIMAL_FOLLOWERS_02.19.2024__17.01.30__START0_END-1_uofmisinfowatch_acresearcher.csv")

PRINTING STATS
Total N edges: 502760
Followers by spreader:
           main  followers_id
2     exthepose          7995
1  RealRawNews1         46419
3  redvoicenews         48346
0  JackPosobiec        200000
4      stkirsch        200000

Splitting into treatment and control with treat prob = 0.8
Verify logic works:
main          group    
JackPosobiec  treatment    0.799510
              control      0.200490
RealRawNews1  treatment    0.800900
              control      0.199100
exthepose     treatment    0.801376
              control      0.198624
redvoicenews  treatment    0.800728
              control      0.199272
stkirsch      treatment    0.799510
              control      0.200490
Name: group, dtype: float64

Raw counts
main          group    
JackPosobiec  treatment    159902
              control       40098
RealRawNews1  treatment     37177
              control        9242
exthepose     treatment      6407
              control        1588
redvoicenews  treatment     